refer to this for installation https://github.com/abetlen/llama-cpp-python#installation-with-openblas--cublas--clblast--metal 

In [1]:
%pip install llama_index
%pip install llama-cpp-python
%pip install transformers
%pip install torch
%pip install tqdm


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging
import sys

from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)  # Change INFO to DEBUG if you want more extensive logging
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

llm = LlamaCPP(
    model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf",
    
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    
    temperature=0.0,
    max_new_tokens=1024,
    
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,  # note, this sets n_ctx in the model_kwargs below, so you don't need to pass it there.
    
    # kwargs to pass to __call__()
    generate_kwargs={},
    
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 4}, # I need to play with this and see if it actually helps
    
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [3]:
# Create an index of your documents
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

storage_directory = "./storage"

documents = SimpleDirectoryReader('./testdata').load_data()

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)


index = VectorStoreIndex.from_documents(documents, service_context=service_context)
# Persist the index to disk
index.storage_context.persist(persist_dir=storage_directory)

**********
Trace: index_construction
    |_node_parsing ->  0.00599 seconds
      |_chunking ->  0.001005 seconds
      |_chunking ->  0.000992 seconds
      |_chunking ->  0.001004 seconds
      |_chunking ->  0.000995 seconds
      |_chunking ->  0.000998 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.0 seconds
    |_embedding ->  0.52004 seconds
**********


In [4]:
# Now you can load the index from disk when needed, and not rebuild it each time.
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index import StorageContext, load_index_from_storage

# transcript_directory = "transcripts/ancient-aliens-official"
storage_directory = "./storage"

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)

storage_context = StorageContext.from_defaults(persist_dir=storage_directory)
index = load_index_from_storage(storage_context, service_context=service_context)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
**********
Trace: index_construction
**********


In [5]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

# response = query_engine.query("What do you think of Facebook's LLaMa?")
# print(response)

# query = "What is the Alien nuclear agenda? \n\n" \
#         "Please summarize the information into 3 detailed paragraphs, something suitable for a blog post."

query = "could you combine the learning styles from the document and generate a few game types based on the octalysis framework for different types of games"

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

**********
Trace: query
    |_query ->  650.275208 seconds
      |_retrieve ->  0.025978 seconds
        |_embedding ->  0.024977 seconds
      |_synthesize ->  650.247467 seconds
        |_templating ->  0.0 seconds
        |_llm ->  650.213329 seconds
**********


<b>  Certainly! Based on the provided context information, I can help you combine the learning styles from the document and generate a few game types based on the Octalysis framework for different types of games.

Here are some game ideas that incorporate multiple learning styles and the Octalysis framework:

1. Virtual Reality (VR) Exploration Games for Visual, Kinesthetic, and Naturalistic Learners:
	* Game Idea: "Eco-Warriors VR" - A nature-based VR game where players explore a virtual ecosystem, restore balance, and defend against threats. Players can interact with the environment using hand gestures or voice commands, and receive feedback through visual cues and sounds.
2. Strategy-based Simulation Games for Logical, Kinesthetic, and Solitary Learners:
	* Game Idea: "City Builder" - A city-building simulation game where players manage resources, plan developments, and address real-time challenges to grow their city. Players can use logical reasoning to solve problems, manipulate objects in the virtual space using gestures or mouse clicks, and receive feedback through visual progress indicators.
3. Rhythm and Sound Puzzle Games for Auditory, Kinesthetic Learners:
	* Game Idea: "Beat Blaster" - A music-based puzzle game where players match beats and rhythms to succeed. Players can use their sense of hearing and timing to complete levels, and receive feedback through sound effects and visual cues.
4. AR Outdoor Quests for Naturalistic, Social, and Kinesthetic Learners:
	* Game Idea: "Wildlife Explorer" - An augmented reality game where players embark on outdoor quests to discover virtual objects in nature, collaborate with others in real-time, and compete in environmental challenges. Players can use their sense of exploration and collaboration to succeed, and receive feedback through AR visuals and sounds.
5. Trivia and Quiz Challenges for Read/Write, Social, and Logical Learners:
	* Game Idea: "QuizUp" - A trivia game that challenges players to answer questions, compete in teams, and solve complex puzzles. Players can use their reading comprehension skills to absorb information, logical reasoning to solve problems, and social skills to collaborate with others. Feedback is provided through visual progress indicators, sounds, and leaderboards.

These game ideas incorporate multiple learning styles and the Octalysis framework to create engaging and well-rounded learning experiences. By combining different game mechanics and motivational drivers, these games can cater to a wide range of learners and preferences.</b>

In [6]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

# response = query_engine.query("What do you think of Facebook's LLaMa?")
# print(response)

# query = "What is the Alien nuclear agenda? \n\n" \
#         "Please summarize the information into 3 detailed paragraphs, something suitable for a blog post."

query = "I am trying to develop a game to enhance adult learning learning \n\n" \
        "Given that the game is a narrative driven adventure game, please generate 5 multiple choice question regarding sexual harrassment \n\n"

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit


**********
Trace: query
    |_query ->  643.355441 seconds
      |_retrieve ->  0.601479 seconds
        |_embedding ->  0.546591 seconds
      |_synthesize ->  642.753737 seconds
        |_templating ->  0.0 seconds
        |_llm ->  642.674986 seconds
**********


<b>  Sure, I'd be happy to help! Based on the context information provided, here are five multiple-choice questions related to sexual harassment that could be incorporated into a narrative-driven adventure game for adult learning:

1. What is considered appropriate behavior in a professional setting to avoid being perceived as sexually harassing someone?
a) Being overly flirtatious and making explicit comments
b) Asking about someone's personal life or relationships
c) Offering to help with tasks or providing support without expectation of reciprocity
d) All of the above are appropriate behaviors

Answer: c) Offering to help with tasks or providing support without expectation of reciprocity.

2. Which of the following is an example of quid pro quo sexual harassment?
a) A supervisor offering a subordinate a promotion in exchange for a date
b) A coworker making explicit comments to a colleague in the break room
c) A manager asking an employee out on a date and being rejected
d) All of the above are examples of quid pro quo sexual harassment

Answer: a) A supervisor offering a subordinate a promotion in exchange for a date.

3. What is the purpose of the Title IX legislation in the United States?
a) To prohibit discrimination based on gender in education
b) To protect employees from sexual harassment in the workplace
c) To provide legal recourse for victims of sexual harassment
d) All of the above

Answer: a) To prohibit discrimination based on gender in education.

4. What is an example of a hostile work environment?
a) A coworker making offensive comments about someone's appearance
b) A supervisor who consistently gives unfair performance reviews to one employee
c) A workplace that is physically unsafe or has hazardous conditions
d) All of the above are examples of a hostile work environment

Answer: d) All of the above are examples of a hostile work environment.

5. What should an employee do if they experience or witness sexual harassment in the workplace?
a) Ignore the behavior and hope it stops
b) Report the behavior to HR or a supervisor
c) Talk to the person engaging in the behavior and try to resolve the issue
d) All of the above are appropriate actions

Answer: b) Report the behavior to HR or a supervisor.</b>